# AOC 2021

Welcome to the Advent of Code 2021 !

## Basic configuration



In [ ]:
# help for aocd : https://pypi.org/project/advent-of-code-data/

#!pip install aocd

In [ ]:
import os

#replace by your login session cookie
os.environ['AOC_SESSION'] = "53616c7465645f5f0ad0842a2df630f5dd756f39b35305c5fc68cb931c932f61b2afcd0b570c5c78a80003c9ab0409de" #your login session cookie

In [ ]:
from aocd import submit
from aocd.models import Puzzle

In [ ]:
import os 
import numpy as np
from tqdm import tqdm
import json
import typing as tp
from collections import Counter
import math

## Day 24
https://adventofcode.com/2021/day/24
### Prepare input

In [ ]:
puzzle = Puzzle(year=2021, day=24)

### Part 1

### Part 2

## Day 23
https://adventofcode.com/2021/day/23
### Prepare input

In [ ]:
puzzle = Puzzle(year=2021, day=23)

### Part 1

### Part 2

## Day 22
https://adventofcode.com/2021/day/22
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=22)

### Part 1

### Part 2

## Day 21
https://adventofcode.com/2021/day/21
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=21)

### Part 1

### Part 2

## Day 20
https://adventofcode.com/2021/day/20
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=20)


### Part 1

### Part 2

## Day 19
https://adventofcode.com/2021/day/19
### Prepare input

In [ ]:
puzzle = Puzzle(year=2021, day=19)

### Part 1

### Part 2

## Day 18
https://adventofcode.com/2021/day/18
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=18)

### Part 1


### Part 2

## Day 17
https://adventofcode.com/2021/day/17
### Prepare input

In [ ]:
puzzle = Puzzle(year=2021, day=17)

### Part 1

### Part 2

## Day 16
https://adventofcode.com/2021/day/16
### Prepare input

In [ ]:
puzzle = Puzzle(year=2021, day=16)


### Part 1

### Part 2

## Day 15
https://adventofcode.com/2021/day/15
### Prepare input

In [ ]:
puzzle = Puzzle(year=2021, day=15)

### Part 1

### Part 2

## Day 14
https://adventofcode.com/2021/day/14
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=14)

### Part 1

### Part 2

## Day 13
https://adventofcode.com/2021/day/13
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=13)

### Part 1

### Part 2

## Day 12
https://adventofcode.com/2021/day/12
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=12)

### Part 1

### Part 2

## Day 11
https://adventofcode.com/2021/day/11
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=11)
content = [[int(char) for char in line] for line in puzzle.input_data.split("\n")]

### Part 1

In [ ]:
def get_adjacents(h:int, w:int, idx: int, jdx: int) -> tp.List[tp.Tuple[int, int]]:
    
    adjacents = []
    
    if idx>0:
        adjacents.append((idx-1, jdx))
    if idx<h-1: 
        adjacents.append((idx+1, jdx))
    if jdx>0:
        adjacents.append((idx, jdx-1))
    if jdx<w-1:
        adjacents.append((idx, jdx+1))
        
    if idx>0 and jdx>0: 
        adjacents.append((idx-1,jdx-1))
    if idx>0 and jdx<w-1: 
        adjacents.append((idx-1,jdx+1))
    if idx<h-1 and jdx>0: 
        adjacents.append((idx+1,jdx-1))
    if idx<h-1 and jdx<w-1:
        adjacents.append((idx+1,jdx+1))
           
    return adjacents

In [ ]:
def check(n_flashes:int, levels: np.array, point:tp.Tuple[int, int]): 
        
    levels[point] = 0
    n_flashes += 1
    
    adjacents = get_adjacents(levels.shape[0],levels.shape[1],point[0],point[1])
    for adj in adjacents: 
        if levels[adj] not in {0,10}:
            levels[adj] += 1
            if levels[adj] == 10: 
                n_flashes, levels = check(n_flashes, levels, adj)
                
    return n_flashes, levels

In [ ]:
def one_step(levels:np.array) -> tp.Tuple[int, np.array]:
    h,w=levels.shape
    n_flashes = 0
    
    levels += 1
    
    for y in range(h): 
        for x in range(w):
            if levels[y,x] == 10: 
                cur_flashes, cur_levels = check(n_flashes, levels, (y,x))
                n_flashes = cur_flashes
                levels = cur_levels
            
    return n_flashes, levels

In [ ]:
levels = np.array(content)
tot_flashes = 0
for _ in range(100): 
    n_flashes, levels = one_step(levels)
    tot_flashes += n_flashes
    
tot_flashes

In [ ]:
puzzle.answer_a = tot_flashes

### Part 2

In [ ]:
levels = np.array(content)
count = 0
while not all(levels[y,x]==0 for y in range(levels.shape[0]) for x in range(levels.shape[1])):
    _, levels = one_step(levels)
    count += 1

count 

In [ ]:
puzzle.answer_b = count

## Day 10
https://adventofcode.com/2021/day/10
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=10)
content = puzzle.input_data.split("\n")

### Part 1

In [ ]:
from collections import deque

In [ ]:
corrupted = []
for idx, line in enumerate(content): 
    cur_chunk = deque()
    for jdx,char in enumerate(line): 
        if char in {"(", "[","{","<"}: 
            cur_chunk.append(char)
            continue
        if char in {")", "]", "}", ">"}:
            last_char = cur_chunk.pop()
            if f"{last_char}{char}" not in {"()","[]", "{}", "<>"}:
                corrupted.append((idx, line, char))
                break 

In [ ]:
points = 0
for _,_,char in corrupted: 
    if char == ")":
        points += 3
    elif char == "]": 
        points += 57
    elif char == "}": 
        points += 1197
    else: 
        points += 25137

In [ ]:
points

In [ ]:
puzzle.answer_a = points

### Part 2

In [ ]:
new_content = set(content) - set([line for _, line, _ in corrupted])

In [ ]:
open_sequences = []
for idx, line in enumerate(new_content): 
    cur_chunk = deque()
    for jdx,char in enumerate(line): 
        if char in {"(", "[","{","<"}: 
            cur_chunk.append(char)
        elif char in {")", "]", "}", ">"}:
            last_char = cur_chunk.pop()
    open_sequences.append(cur_chunk)

In [ ]:
closing_sequences = []
for seq in open_sequences: 
    close = []
    while seq: 
        cur_char = seq.pop()
        if cur_char == "(":
            close.append(")")
        elif cur_char == "[":
            close.append("]")
        elif cur_char == "{":
            close.append("}")
        else:
            close.append(">")
    
    closing_sequences.append(close)
    

In [ ]:
total_scores = []
for seq in closing_sequences: 
    score = 0
    for char in seq: 
        score *= 5
        if char == ")":
            score += 1
        elif char == "]": 
            score += 2
        elif char == "}": 
            score += 3
        else: 
            score += 4
    total_scores.append(score)

In [ ]:
middle_idx = len(total_scores) // 2

In [ ]:
total_scores.sort()

In [ ]:
answ = total_scores[middle_idx]
answ

In [ ]:
puzzle.answer_b = answ

## Day 9
https://adventofcode.com/2021/day/9
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=9)
content = puzzle.input_data.split("\n")

### Part 1

In [ ]:
def get_adjacents(content: tp.Sequence[str], idx: int, jdx: int) -> tp.List[tp.Tuple[str, int, int]]:
    
    adjacents = []
    
    if idx>0: 
        adjacents.append((content[idx-1][jdx], idx-1, jdx))
    if idx<len(content)-1: 
        adjacents.append((content[idx+1][jdx], idx+1, jdx))
    if jdx>0:
        adjacents.append((content[idx][jdx-1], idx, jdx-1))
    if jdx<len(content[0])-1:
        adjacents.append((content[idx][jdx+1], idx, jdx+1))
    
    return adjacents

In [ ]:
low = []
for idx, line in enumerate(content): 
    for jdx, point in enumerate(line): 
        adjacents = get_adjacents(content, idx, jdx)

        adj_values = list(int(adj[0]) for adj in adjacents)
        if all(x>int(point) for x in adj_values): 
            low.append([point, idx, jdx])

In [ ]:
answ = sum(int(pt[0])+1 for pt in low)
answ

In [ ]:
submit(answ, part="a", day=9, year=2021) 

### Part 2

In [ ]:
def get_bassin_size(point: tp.Tuple[str, int, int], seen: tp.Optional[tp.Set]=None) -> int: 
    if seen is None: 
        seen = set()
    _, idx, jdx = point
    
    adjacents = get_adjacents(content, idx, jdx)
    
    size = 0
    for adj in adjacents: 
        if adj not in seen and int(adj[0]) != 9: 
            seen.add(adj)
            size += 1 + get_bassin_size(adj, seen)
    
    return size

In [ ]:
bassin_sizes = []

for point in low: 
    bassin_sizes.append(get_bassin_size(point))
    
answ = math.prod(sorted(bassin_sizes)[-3:])
answ

In [ ]:
submit(answ, part="b", day=9, year=2021) 

## Day 8
https://adventofcode.com/2021/day/8
### Prepare input 

### Part 1

In [ ]:
puzzle = Puzzle(year=2021, day=8)
content = [x.split(" | ") for x in puzzle.input_data.split("\n")]
content = [[x1.split(" "), x2.split(" ")] for x1, x2 in content]

### Part 1

In [ ]:
n_segments = {0:6, 1:2, 2:5, 3:5, 4:4, 5:5, 6:6, 7:3, 8:7, 9:6}

In [ ]:
answ = 0
for _, output in content: 
    for digits in output: 
        if len(digits) in {n_segments[1], n_segments[4], n_segments[7], n_segments[8]}:
            answ += 1
answ

In [ ]:
submit(answ, part="a", day=8, year=2021) 

### Part2

In [ ]:
def decode(patterns: tp.Sequence[str]) -> tp.Dict[tp.FrozenSet, int]:
    
    positions = {}
    solutions = {}
    
    solutions[1] = [el for el in patterns if len(el) == 2][0]
    solutions[7] = [el for el in patterns if len(el) == 3][0]
    solutions[4] = [el for el in patterns if len(el) == 4][0]
    solutions[8] = [el for el in patterns if len(el) == 7][0]
    
    positions["t"] = set(solutions[7]) - set(solutions[1])
    
    solutions[9] = [el for el in patterns if len(el) == 6 and len(set(el) - set(solutions[4])) == 2][0]
    positions["b"] = set(solutions[9]) - set(solutions[4]) - positions["t"]
    
    solutions[0] = [el for el in patterns if len(el) == 6 and el != solutions[9] and set(solutions[7]) <= set(el)][0]
    solutions[6] = [el for el in patterns if len(el) == 6 and el != solutions[9] and el != solutions[0]][0]
    
    positions["m"] = set(solutions[6]) - set(solutions[0])
    positions["tr"] = set(solutions[0]) - set(solutions[6])
    positions["br"] = set(solutions[1]) - positions["tr"]
    
    solutions[5] = [el for el in patterns if len(el) == 5 and not (positions["tr"] <= set(el))][0]
    solutions[3] = [el for el in patterns if len(el) == 5 and el != solutions[5] and positions["br"] <= set(el)][0]
    solutions[2] = [el for el in patterns if len(el) == 5 and el != solutions[5] and el != solutions[3]][0]
    
    return {frozenset(v): k for k, v in solutions.items()}
    

In [ ]:
answ = 0
for entry in content: 
    patterns, outputs = entry
    
    code = decode(patterns)
    
    result = 0
    for out in outputs:
        result = result*10 + code[frozenset(out)]
    answ += result

answ

In [ ]:
submit(answ, part="b", day=8, year=2021) 

## Day 7
https://adventofcode.com/2021/day/7
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=7)
content = list(map(int, puzzle.input_data.split(",")))

### Part 1

In [ ]:
from statistics import median, mean

In [ ]:
med_pos = int(median(content))

def dist(pos: int) -> int: 
    return abs(pos-med_pos)

answ = sum(map(dist, content))
answ

In [ ]:
submit(answ, part="a", day=7, year=2021) 

### Part 2

In [ ]:
mean_pos = int(mean(content))

def dist2(pos:int) -> int: 
    n = abs(pos - mean_pos)
    return n * (n + 1) // 2 

answ = sum(map(dist2, content))
answ

In [ ]:
submit(answ, part="b", day=7, year=2021) 

## Day 6
https://adventofcode.com/2021/day/6
### Prepare input

In [ ]:
puzzle = Puzzle(year=2021, day=6)
content = list(map(int, puzzle.input_data.split(",")))

### Part 1

In [ ]:
def iterate(cur_count: Counter) -> Counter: 
    new_count = Counter() 
    for timer in range(1,9): 
        new_count[timer-1] = cur_count[timer]
    new_count[6] += cur_count[0]
    new_count[8] += cur_count[0]
    
    return new_count

In [ ]:
count = Counter(content)
for _ in range(80):
    count = iterate(count)

In [ ]:
answ = sum(count.values())
answ

In [ ]:
submit(answ, part="a", day=6, year=2021) 

### Part 2

In [ ]:
count = Counter(content)
for _ in range(256): 
    count = iterate(count)
    
answ = sum(count.values())
answ

In [ ]:
submit(answ, part="b", day=6, year=2021) 

## Day 5
https://adventofcode.com/2021/day/5
### Prepare input

In [ ]:
puzzle = Puzzle(year=2021, day=5)

In [ ]:
content = puzzle.input_data.split("\n")

In [ ]:
class Diagram:
    def __init__(self, incontent: tp.Sequence[str]) -> None: 
        
        self.segments = [[tuple(map(int, row.split(","))) for row in line.split(" -> ")] for line in incontent]
            
        self.rows = self.get_rows()
        self.cols = self.get_cols()
        self.diags = self.get_diags()
        
        self.counter = Counter()
        
    def get_rows(self) -> tp.List[tp.List[tp.Tuple[int,int]]]: 
        return [segm for segm in self.segments if segm[0][0] == segm[1][0]]
    
    def get_cols(self) -> tp.List[tp.List[tp.Tuple[int,int]]]: 
        return [segm for segm in self.segments if segm[0][1] == segm[1][1]]
    
    def get_diags(self) -> tp.List[tp.List[tp.Tuple[int,int]]]: 
        return [segm for segm in self.segments if segm[0][0] != segm[1][0] and segm[0][1] != segm[1][1]]
        
    def count_rows(self) -> None: 
        for (x1,y1),(x2,y2) in self.rows:
            dy = 1 if y1 < y2 else -1
            self.counter.update([(x1, y) for y in range(y1, y2 + dy, dy)])
 
    def count_cols(self) -> None:
        for (x1,y1),(x2,y2) in self.cols:
            dx = 1 if x1 < x2 else -1
            self.counter.update([(x, y1) for x in range(x1, x2 + dx, dx)])
                
    def count_diags(self) -> None:
        for (x1,y1),(x2,y2) in self.diags:
            dx = 1 if x1 < x2 else -1
            dy = 1 if y1 < y2 else -1
            self.counter.update([(x, y) for x, y in zip(range(x1, x2 + dx, dx), range(y1, y2 + dy, dy))])
            
    def overlap(self, part: int=1) -> int: 
        self.reset_counter()
        
        self.count_rows()
        self.count_cols() 
        if part == 2:
            self.count_diags()
        
        return len([point for point, count in self.counter.items() if count >= 2])
        
    def reset_counter(self) -> None:
        self.counter = Counter()
        

### Part 1

In [ ]:
diagram = Diagram(content)

In [ ]:
answ=diagram.overlap()
answ

In [ ]:
submit(answ, part="a", day=5, year=2021) 

### Part 2

In [ ]:
answ=diagram.overlap(2)
answ

In [ ]:
submit(answ, part="b", day=5, year=2021)

## Day 4
https://adventofcode.com/2021/day/4
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=4)

In [ ]:
content = puzzle.input_data.split("\n\n")

In [ ]:
numbers = [int(x) for x in content[0].split(",")]

In [ ]:
class Board: 
    def __init__(self, inboard: tp.Sequence[tp.Sequence[int]]) -> None: 
        self.board = np.array(inboard)
        self.rows = [set(r) for r in self.board]
        self.cols = [set(c) for c in np.transpose(self.board)]
    
    def sum_leftover(self) -> int: 
        return sum(sum(r) for r in self.rows)
    
    def discard_nb(self, nb: int) -> None: 
        self.rows = [r-{nb} for r in self.rows]
        self.cols = [c-{nb} for c in self.cols]
    
    def is_winner(self) -> bool: 
        return not (all(self.rows) and all(self.cols))

In [ ]:
def set_boards(boards_content: tp.Sequence[str]) -> tp.List[Board]: 
    return [Board([[int(n) for n in row.split()] for row in line.split("\n")]) for line in boards_content]

In [ ]:
boards = set_boards(content[1:])

success_rate = []
for bdx, board in enumerate(boards): 
    for ndx, nb in enumerate(numbers): 
        board.discard_nb(nb)
        if board.is_winner(): 
            success_rate.append([ndx, nb * board.sum_leftover()])
            break
            
success_rate.sort(key = lambda x: x[0])

### Part 1

In [ ]:
answ = success_rate[0][1]
answ

In [ ]:
submit(answ, part="a", day=4, year=2021)

### Part 2

In [ ]:
answ = success_rate[-1][1]
answ

In [ ]:
submit(answ, part="b", day=4, year=2021)

## Day 3
https://adventofcode.com/2021/day/3
### Prepare input

In [ ]:
puzzle = Puzzle(year=2021, day=3)
content = puzzle.input_data.split("\n")

In [ ]:
content

### Part 1

In [ ]:
def transpose(inlist: tp.Sequence[str]) -> tp.List[str]: 
    return ["".join(s) for s in zip(*inlist)]

In [ ]:
def bin2int(binary: tp.Union[str, tp.Sequence[str]]) -> int: 
    if isinstance(binary, list): 
        return int("".join(binary),2)
    return int(binary, 2)

In [ ]:
counters = [Counter(pos) for pos in transpose(content)]

In [ ]:
gamma = ["0" if c["0"]>c["1"] else "1" for c in counters]
eps = ''.join(str(1-int(x)) for x in gamma)

In [ ]:
answ = bin2int(gamma) * bin2int(eps)
answ

In [ ]:
submit(answ, part="a", day=3, year=2021)

### Part 2

In [ ]:
def keep(inlist: tp.Sequence[str], bpos: int, bval: str) -> tp.List[str]: 
    return [el for el in inlist if el[bpos] == bval]

In [ ]:
def apply_criteria(init: tp.Sequence[str], default: int) -> int: 
    out = init.copy()
    idx = 0
    while len(out) > 1: 
        counter= Counter(transpose(out)[idx])
        if counter["0"]>counter["1"]:
            out = keep(out, idx, str(1-default))
        else: 
            out = keep(out, idx, str(default))
        idx += 1
    return bin2int(out[0])

In [ ]:
oxygen = apply_criteria(content, 1)
co2 = apply_criteria(content, 0)
anws = oxygen * co2
answ

In [ ]:
submit(answ, part="b", day=3, year=2021)

## Day 2
https://adventofcode.com/2021/day/2
### Prepare input

In [ ]:
puzzle = Puzzle(year=2021, day=2)

In [ ]:
content = [x.split(" ") for x in puzzle.input_data.split("\n")]

In [ ]:
content

### Part 1

In [ ]:
h_pos = 0
depth = 0
for step, x in content: 
    x = int(x)
    
    if step == "forward": 
        h_pos += x
    elif step == "down": 
        depth += x
    elif step == "up": 
        depth -= x
        
answ = h_pos * depth
answ

In [ ]:
submit(answ, part="a", day=2, year=2021)

### Part 2

In [ ]:
h_pos = 0
depth = 0
aim = 0
for step, x in content: 
    x = int(x)
    
    if step == "forward": 
        h_pos += x
        depth += (aim * x)
    elif step == "down": 
        aim += x
    elif step == "up": 
        aim -= x

print(h_pos, depth, aim)
answ = h_pos * depth
answ

In [ ]:
submit(answ, part="b", day=2, year=2021)

## Day 1
https://adventofcode.com/2021/day/1

### Prepare input

In [ ]:
puzzle = Puzzle(year=2021, day=1)

In [ ]:
content = [int(x) for x in puzzle.input_data.split("\n")]

### Part 1

In [ ]:
def count_increase(inlist: tp.Sequence[int]) -> int: 
    count = 0
    for i in range(len(inlist)-1): 
        if inlist[i] < inlist[i+1]:
            count += 1
            
    return count

In [ ]:
answ = count_increase(content)
answ

In [ ]:
submit(answ, part="a", day=1, year=2021)

### Part 2

In [ ]:
sums = [sum(content[i:i+3]) for i in range(len(content)-2)]

In [ ]:
answ = count_increase(sums)
answ

In [ ]:
submit(answ, part="b", day=1, year=2021)